In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from tqdm import tqdm
import re
import configparser

In [2]:
cofig = configparser.RawConfigParser()
cofig.read('config.ini')
url = cofig['scrap_url']['url']

In [3]:
def open_browser(path):
    try:
        driver = webdriver.Chrome()
        driver.get(path)

        time.sleep(10) # Wait for page to load
        
        driver.find_element(By.XPATH, '//button[text() ="Ask Me Later"]').click()

        print('Driver created successfully')
        return driver
    except Exception as e:
        raise e
    
def collect_data(driver, scroll_value=1100):
    try:
        product_data = []  # Store results

        # Scroll multiple times to load more images
        for _ in tqdm(range(scroll_value)):  
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
            time.sleep(4)  # Allow time for new images to load

            # Find all product links
            product_cards = driver.find_elements(By.CLASS_NAME, "ProductModule__aTag")

            for product in product_cards:
                # Extract the product name from the title attribute
                product_name = product.get_attribute("title")

                try:
                    # Find the image element inside the product card
                    img_tag = product.find_element(By.TAG_NAME, "img")
                    img_url = img_tag.get_attribute("src")

                    # TataCliq sometimes provides relative URLs; make them absolute
                    if img_url.startswith("//"):
                        img_url = "https:" + img_url
                except:
                    img_url = "No Image Found"

                # Store the extracted details
                product_data.append((product_name, img_url))

        # Close the browser
        driver.quit()
        return product_data
    except Exception as e:
        raise e

In [4]:
driver = open_browser(path=url)
product_data = collect_data(driver)

Driver created successfully


100%|██████████| 1100/1100 [1:37:33<00:00,  5.32s/it]


In [5]:
scrap_data = pd.DataFrame(product_data, columns=['product_name', 'image_url'])
image_data = scrap_data[~(scrap_data['image_url']=='No Image Found')]

pattern = r'MP\d+'
image_data['product_id'] = image_data['image_url'].apply(lambda x: re.findall(pattern, x)[0])

/tmp/ipykernel_56522/561394997.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_data['product_id'] = image_data['image_url'].apply(lambda x: re.findall(pattern, x)[0])


In [8]:

image_data.to_csv('product_table.csv', index=False)